# `BUILDING A REGRESSION MODEL TO FORECAST SALES` 
#### Using the CRISP-DM framework

## `Business Understanding`


#### Goal/Description
To create a machine learning model to forecast the demand of products in various locations.

#### `Null Hypothesis`
There is no relationship between the average sales and the average sales of the various branches

#### `Alternate Hypothesis`
There is a relationship between the average sales and the average sales of the various branches

### `Key Metrics and Success Criteria`

The success of this poject will be evaluated based on several key metrics and success criteria including;





#### `Analytical Questions`

Is the train dataset complete (has all the required dates)?
Which dates have the lowest and highest sales for each year (excluding days the store was closed)?
Compare the sales for each month across the years and determine which month of which year had the highest sales.
Did the earthquake impact sales?
Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
Are sales affected by promotions, oil prices and holidays?
What analysis can we get from the date and its extractable features?
Which product family and stores did the promotions affect.
What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.

## `Data Understanding`

#### Data Source
The data was sourced from three places:
- Three tables from Corporation Favorita's database 
- Two csv files from a OneDrive location
- Two csv files from a GitHub repository 
 

#### Data Features

- store_nbr : Identifies the store at which the products are sold.

- family : identifies the type of product sold.

- sales : gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

- onpromotion : gives the total number of items in a product family that were being promoted at a store at a given date.

#### Data Exploration

##### `Libraries`

In [137]:
#Libraries imported
import sqlalchemy as sa
import pyodbc  
from dotenv import dotenv_values 
import pandas as pd
from scipy import stats 
from scipy.stats import kruskal
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import collections
import numpy as np
import warnings

##### `Database Connection`

In [138]:
#Access protocols for the SQL Database
env_variables = dotenv_values('logins.env')
database = env_variables.get('database')
server = env_variables.get('server')
username = env_variables.get('username')
password = env_variables.get('password')

Python-dotenv could not parse statement starting at line 5


In [139]:
#Creation of Connection to Database
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"
connection = pyodbc.connect(connection_string)

In [140]:
#Querying SQL Database and reading the table into a dataframe
query = "SELECT * FROM oil"

oil_df = pd.read_sql(query, connection,parse_dates= 'date')#,index_col=['date'])

oil_df.head(5)

C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\1268661856.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [141]:
#Querying SQL Database and reading the table into a dataframe
query = "SELECT * FROM holidays_events"

hdays_df = pd.read_sql(query, connection,parse_dates='date')

hdays_df.head(5)

C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\527256426.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [142]:
#Querying SQL Database and reading the table into a dataframe
query = "SELECT * FROM stores"

stores_df= pd.read_sql(query, connection)
stores_df.head(10)

C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\289228698.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [156]:
##Accessing the csv data 
sales_df = pd.read_csv("data\\store-sales-forecasting\\train.csv",parse_dates = ['date'])
sales_df.tail(10)

,id,date,store_nbr,family,sales,onpromotion
3000878,3000878,2017-08-15,9,MAGAZINES,11.000,0
3000879,3000879,2017-08-15,9,MEATS,449.228,0
3000880,3000880,2017-08-15,9,PERSONAL CARE,522.000,11
3000881,3000881,2017-08-15,9,PET SUPPLIES,6.000,0
3000882,3000882,2017-08-15,9,PLAYERS AND ELECTRONICS,6.000,0
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


In [144]:
##Accessing the csv data 
transact_df = pd.read_csv("data\\store-sales-forecasting\\transactions.csv",index_col='date',parse_dates = ['date'])
transact_df.head(10)


,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922
2013-01-02,5,1903
2013-01-02,6,2143
2013-01-02,7,1874
2013-01-02,8,3250


### EDA

In [157]:
### finding missing days in the sales data
sales_df.isnull().sum()

start_day = sales_df.date.min()
end_day = sales_df.date.max()

print(f'{start_day} \n\n {end_day}')

existing_date = pd.date_range(start=start_day,end = end_day,freq="D")

missing_dates = existing_date[~existing_date.isin(sales_df.index)]
print(missing_dates)

2013-01-01 00:00:00 

 2017-08-15 00:00:00
DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', length=1688, freq='D')


#Lets look for trends,seasonality and cyclic in each df


In [159]:
# Function to get the day name from a date string
from dateutil.parser import parse

def get_day_name(date_str):
    return date_str.strftime('%A')


day_names = []
# Get day names for each specific date
for date in missing_dates:
    day = get_day_name(date)
    day_names.append(day)


print("Day names for specific dates:")
for date, day_name in zip(missing_dates, day_names):
    print(f"{date}: {day_name}")

# Create a DataFrame for missing dates
missing_dates_df = pd.DataFrame({'date': pd.to_datetime(missing_dates)})
    

Day names for specific dates:
2013-01-01 00:00:00: Tuesday
2013-01-02 00:00:00: Wednesday
2013-01-03 00:00:00: Thursday
2013-01-04 00:00:00: Friday
2013-01-05 00:00:00: Saturday
2013-01-06 00:00:00: Sunday
2013-01-07 00:00:00: Monday
2013-01-08 00:00:00: Tuesday
2013-01-09 00:00:00: Wednesday
2013-01-10 00:00:00: Thursday
2013-01-11 00:00:00: Friday
2013-01-12 00:00:00: Saturday
2013-01-13 00:00:00: Sunday
2013-01-14 00:00:00: Monday
2013-01-15 00:00:00: Tuesday
2013-01-16 00:00:00: Wednesday
2013-01-17 00:00:00: Thursday
2013-01-18 00:00:00: Friday
2013-01-19 00:00:00: Saturday
2013-01-20 00:00:00: Sunday
2013-01-21 00:00:00: Monday
2013-01-22 00:00:00: Tuesday
2013-01-23 00:00:00: Wednesday
2013-01-24 00:00:00: Thursday
2013-01-25 00:00:00: Friday
2013-01-26 00:00:00: Saturday
2013-01-27 00:00:00: Sunday
2013-01-28 00:00:00: Monday
2013-01-29 00:00:00: Tuesday
2013-01-30 00:00:00: Wednesday
2013-01-31 00:00:00: Thursday
2013-02-01 00:00:00: Friday
2013-02-02 00:00:00: Saturday
2013-0

In [160]:
### Adding the missing dates to the dataframe
# Merge with specific dates to include missing dates
sales_df = sales_df.reset_index(drop=False)
sales_df = pd.concat([sales_df, missing_dates_df], ignore_index=True)
sales_df

,index,id,date,store_nbr,family,sales,onpromotion
0,0.0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0
1,1.0,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0
2,2.0,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0
3,3.0,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0
4,4.0,4.0,2013-01-01,1.0,BOOKS,0.0,0.0
...,...,...,...,...,...,...,...
3002571,NaN,NaN,2017-08-11,NaN,NaN,NaN,NaN
3002572,NaN,NaN,2017-08-12,NaN,NaN,NaN,NaN
3002573,NaN,NaN,2017-08-13,NaN,NaN,NaN,NaN
3002574,NaN,NaN,2017-08-14,NaN,NaN,NaN,NaN


In [165]:
### Merging the sales and oil dataframes
combined_df = pd.merge(sales_df, oil_df , on='date', how='left') 
                       

TypeError: merge() missing 1 required positional argument: 'right'

In [167]:
### Merging the sales,holidays and oil dataframes
combined_hso_df = pd.merge(combined_df, hdays_df , on='date', how='left') 
combined_hso_df.head(10)

,index,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred
0,0.0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1.0,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2.0,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3.0,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4.0,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
5,5.0,5.0,2013-01-01,1.0,BREAD/BAKERY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
6,6.0,6.0,2013-01-01,1.0,CELEBRATION,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
7,7.0,7.0,2013-01-01,1.0,CLEANING,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
8,8.0,8.0,2013-01-01,1.0,DAIRY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
9,9.0,9.0,2013-01-01,1.0,DELI,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [168]:
combined_ts_df = pd.merge(combined_df, hdays_df , on='date', how='left') 
combined_ts_df.head(10)


,index,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred
0,0.0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1.0,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2.0,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3.0,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4.0,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056061,NaN,NaN,2017-08-11,NaN,NaN,NaN,NaN,48.810001,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
3056062,NaN,NaN,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3056063,NaN,NaN,2017-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3056064,NaN,NaN,2017-08-14,NaN,NaN,NaN,NaN,47.590000,NaN,NaN,NaN,NaN,NaN


In [162]:
### finding missing days in the sales data
transact_df.isnull().sum()

start_day = transact_df.index.min()
end_day = transact_df.index.max()

print(f'{start_day} \n\n {end_day}')

existing_date = pd.date_range(start=start_day,end = end_day,freq="D")

missing_dates = existing_date[~existing_date.isin(transact_df.index)]
print(missing_dates)

2013-01-01 00:00:00 

 2017-08-15 00:00:00
DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-01-01',
               '2016-01-03', '2016-12-25'],
              dtype='datetime64[ns]', freq=None)


In [163]:
day_names = []
# Get day names for each specific date
for date in missing_dates:
    day = get_day_name(date)
    day_names.append(day)


print("Day names for specific dates:")
for date, day_name in zip(missing_dates, day_names):
    print(f"{date}: {day_name}")

# Create a DataFrame for missing dates
missing_dates_df = pd.DataFrame({'date': pd.to_datetime(missing_dates)})
    

Day names for specific dates:
2013-12-25 00:00:00: Wednesday
2014-12-25 00:00:00: Thursday
2015-12-25 00:00:00: Friday
2016-01-01 00:00:00: Friday
2016-01-03 00:00:00: Sunday
2016-12-25 00:00:00: Sunday


### Observation
holidays only

#### Merging the sales,transactions and oil data

In [164]:
# whole_df = pd.merge(combined_df, transact_df , on='date', how='left') 
# whole_df

,index,id,date,store_nbr_x,family,sales,onpromotion,dcoilwtico,store_nbr_y,transactions
0,0.0,0.0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0,NaN,25.0,770.0
1,1.0,1.0,2013-01-01,1.0,BABY CARE,0.0,0.0,NaN,25.0,770.0
2,2.0,2.0,2013-01-01,1.0,BEAUTY,0.0,0.0,NaN,25.0,770.0
3,3.0,3.0,2013-01-01,1.0,BEVERAGES,0.0,0.0,NaN,25.0,770.0
4,4.0,4.0,2013-01-01,1.0,BOOKS,0.0,0.0,NaN,25.0,770.0
...,...,...,...,...,...,...,...,...,...,...
148862669,NaN,NaN,2017-08-15,NaN,NaN,NaN,NaN,47.57,50.0,2804.0
148862670,NaN,NaN,2017-08-15,NaN,NaN,NaN,NaN,47.57,51.0,1573.0
148862671,NaN,NaN,2017-08-15,NaN,NaN,NaN,NaN,47.57,52.0,2255.0
148862672,NaN,NaN,2017-08-15,NaN,NaN,NaN,NaN,47.57,53.0,932.0


In [111]:
missing_dates

DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)

In [109]:
# function to create the year,month and days
def calender(df): 
    try:
        df['monthname'] = df['date'].dt.month_name()
        df['dayname'] = df['date'].dt.day_name()
        df['year'] = df['date'].dt.year
    except KeyError:
        df['date'] = df.index
        df['monthname'] = df['date'].dt.month_name()
        df['dayname'] = df['date'].dt.day_name()
        df['year'] = df['date'].dt.year
    return df

In [68]:
sales_df = calender(sales_df)
transact_df = calender(transact_df)
hdays_df = calender(hdays_df)
oil_df = calender(oil_df)


#### SALES

In [85]:
# Graph of trends
patterns = ['W','ME','YE']

for pattern in patterns:
        #create new DataFrame
    resample_df = pd.DataFrame()

    #create 'sales' column that summarizes total sales by week
    resample_df['sales'] = sales_df['sales'].resample(pattern).sum()
    #plot weekly sales data
    
    fig = px.line(resample_df,x=resample_df.index, y=resample_df.sales) 
    fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list ([
dict(count = 1, label = 'Sect 1' , step = 'year' , stepmode = 'backward'),
dict(count = 2, label = 'Sect 2' , step = 'year' , stepmode = 'backward'),
dict(count = 3, label = 'Sect 3' , step = 'year' , stepmode = 'backward'),
dict(step = 'all')
            ])
        )
    )

    # Creating the Figure instance
 
# showing the plot
    fig.show()



C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\1816509536.py:9: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\1816509536.py:9: FutureWarning:

'A' is deprecated and will be removed in a future version, please use 'YE' instead.



#### Transactions

In [71]:
# Graph of trends
patterns = ['W','M','A']

for pattern in patterns:
        #create new DataFrame
    resample_df = pd.DataFrame()

    #create 'sales' column that summarizes total sales by week
    resample_df['sales'] = transact_df['transactions'].resample(pattern).sum()
    #plot weekly sales data
    
    fig = px.line(x=resample_df.index, y=resample_df.sales) 

    # Creating the Figure instance
 
# showing the plot
    fig.show()

C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\2356674678.py:9: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\2356674678.py:9: FutureWarning:

'A' is deprecated and will be removed in a future version, please use 'YE' instead.



#### Observations

- There is seasonality in the monthly tranactions
- All other graphs are trending

### Checking for gaps in the data

In [86]:
oil = oil_df.groupby('monthname')['dcoilwtico'].mean().sort_values()
oil

monthname
December     61.209884
November     64.062875
March        64.706759
January      65.082079
February     65.727708
August       67.323546
April        68.629231
June         70.277850
May          70.355377
October      70.963034
July         71.243238
September    72.134217
Name: dcoilwtico, dtype: float64

In [92]:
#Checking for missing values
oil_df.isnull().sum()

dcoilwtico    43
date           0
monthname      0
dayname        0
year           0
dtype: int64

In [91]:
#Checking for missing values
transact_df.isnull().sum()

store_nbr       0
transactions    0
date            0
monthname       0
dayname         0
year            0
dtype: int64

In [89]:
#Checking for missing values
sales_df.isnull().sum()

id             0
store_nbr      0
family         0
sales          0
onpromotion    0
date           0
monthname      0
dayname        0
year           0
dtype: int64

In [90]:
#Checking for missing values
hdays_df.isnull().sum()

type           0
locale         0
locale_name    0
description    0
transferred    0
date           0
monthname      0
dayname        0
year           0
dtype: int64

#### Oil

In [78]:
# Graph of trends
patterns = ['W','M','A']

for pattern in patterns:
        #create new DataFrame
    resample_df = pd.DataFrame()

    #create 'sales' column that summarizes total sales by week
    resample_df['sales'] = oil_df['dcoilwtico'].resample(pattern).sum()
    #plot weekly sales data
    
    fig = px.line(x=resample_df.index, y=resample_df.sales) 

    # Creating the Figure instance
 
# showing the plot
    fig.show()

C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\2446099799.py:9: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\chrap\AppData\Local\Temp\ipykernel_8180\2446099799.py:9: FutureWarning:

'A' is deprecated and will be removed in a future version, please use 'YE' instead.



In [75]:
# fig = px.line(sales_df, x=sales_df.index, y="sales")
# fig.show()

In [76]:
# fig = px.line(transact_df, x=transact_df.index, y="transactions")
# fig.show()

In [77]:
# import plotly.graph_objects as go

# # Create random data with numpy
# import numpy as np

# # Create traces
# fig = go.Figure()
# # fig.add_trace(go.Scatter(x=oil_df.date, y=oil_df.dcoilwtico,
# #                     mode='lines',
# #                     name='Oil Price over the years'))
# fig.add_trace(go.Scatter(x=sales_df.index, y=sales_df.sales,
#                     mode='lines+markers',
#                     name='Sales over the years'))
# # fig.add_trace(go.Scatter(x=transact_df.index, y=transact_df.transactions,
# #                     mode='markers', name='markers'))

# fig.show()